In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

AWS_ACCESS_KEY = os.getenv("ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("SECRET_KEY")

WEAVIATE_IP = os.getenv("WEAVIATE_IP")

# Time to Search

## Connect to Weaviate

In [ ]:
import weaviate

client = weaviate.connect_to_custom(
    http_host={WEAVIATE_IP}, http_port="8080",  http_secure=False,
    grpc_host={WEAVIATE_IP}, grpc_port="50051", grpc_secure=False,

    headers={
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
    }
)

In [ ]:
# import weaviate

# client = weaviate.connect_to_local(
#     headers={
#         "X-AWS-Access-Key": AWS_ACCESS_KEY,
#         "X-AWS-Secret-Key": AWS_SECRET_KEY,
#     }
# )

## Vector search
[Docs - near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

In [ ]:
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.near_text(
    query="musical instruments",
    limit=5
)

for item in response.objects:
    print(item.properties)

## Search with filters
[Docs - Filters](https://weaviate.io/developers/weaviate/search/filters)

In [ ]:
from weaviate.classes.query import Filter
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.near_text(
    query="musical instruments",
    limit=5,
    filters=Filter.by_property("category").like("COMMON")
)

for item in response.objects:
    print(item.properties)

## Keyword Search
[Docs - bm25](https://weaviate.io/developers/weaviate/search/bm25)

In [ ]:
response = jeopardy.query.bm25(
    query="railroad",
    # query="railroad africa",
    limit=10,
)

for item in response.objects:
    print(item.properties)

In [ ]:
response = jeopardy.query.bm25(
    query="railroad africa",
    query_properties=["answer"],
    limit=10,
)

for item in response.objects:
    print(item.properties)

## Hybrid search
[Docs - hybrid](https://weaviate.io/developers/weaviate/search/hybrid)

In [ ]:
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.hybrid(
    query="rail, trains and cars",
    alpha=0.8,
    limit=5,
)

for item in response.objects:
    print(item.properties["answer"], " || ", item.properties["question"])

In [ ]:
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.hybrid(
    query="train railroad cars",
    query_properties=["answer^5", "question"],
    alpha=0.5,
    limit=5,
)

for item in response.objects:
    print(item.properties["answer"], " || ", item.properties["question"])

In [ ]:
from weaviate.classes.query import MetadataQuery
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.hybrid(
    query="train railroad cars",
    query_properties=["answer^5", "question"],
    alpha=0.5,
    return_metadata=MetadataQuery(explain_score=True),
    limit=5,
)

for item in response.objects:
    print(item.metadata.explain_score)
    print(item.properties["answer"], " || ", item.properties["question"])

## Close the client when done

In [ ]:
client.close()